In [ ]:
!pip install fastparquet
!pip install pyarrow
!pip install playsound
!pip install pyaudioop
!pip install pydub
!pip install scipy

Load parquet dataset

In [2]:
import pandas as pd

# Load the Parquet dataset
#df = pd.read_parquet('C:/Users/Malek/Downloads/music_dataset')

df = pd.read_parquet('C:/Users/Malek/Downloads/music_dataset_separe/Folk/Folk.parquet')


# Display the first few rows of the dataframe
print(df.head())

                                                  audio  song_id  genre_id  \
5100  {'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...       33         5   
5103  {'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...       37         5   
5117  {'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...       55         5   
5121  {'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...       61         5   
5131  {'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...       74         5   

     genre  
5100  Folk  
5103  Folk  
5117  Folk  
5121  Folk  
5131  Folk  


Describe the data

In [3]:
print(df.info())

# Get descriptive statistics
print(df.describe())

print(df.columns)

<class 'pandas.core.frame.DataFrame'>
Index: 1214 entries, 5100 to 24957
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   audio     1214 non-null   object
 1   song_id   1214 non-null   int64 
 2   genre_id  1214 non-null   int64 
 3   genre     1214 non-null   object
dtypes: int64(2), object(2)
memory usage: 47.4+ KB
None
            song_id  genre_id
count   1214.000000    1214.0
mean   12475.229819       5.0
std     7204.963865       0.0
min       33.000000       5.0
25%     6369.250000       5.0
50%    12537.500000       5.0
75%    18525.500000       5.0
max    24961.000000       5.0
Index(['audio', 'song_id', 'genre_id', 'genre'], dtype='object')


In [ ]:
# @title

from IPython.display import Audio

x=df.iloc[0,0]
y=df.iloc[1,3]

print(x['bytes'])

print(y)

# Assuming `audio` is the variable holding your audio bytes

audio=x['bytes']

# Display the audio

Audio(audio, rate=44100)  # Replace 44100 with the correct sampling rate for your audio data


ça MARCHE !


In [5]:
# @title
from IPython.display import Audio

# Create an Audio object (example with byte data)
audio = Audio(data=audio, rate=44100)

# Save the byte data to a file
with open('/Users/Malek/Downloads/output_audio.wav', 'wb') as f:
    f.write(audio.data)


Enregistrer le son precedent

In [26]:
from IPython.display import Audio

# Extracting values from the DataFrame
x = df.iloc[0, 0]  # Assuming `x` contains a dictionary with the key 'bytes'

y = df.iloc[1, 1]  # Keeping this if needed later

print(y)
print(type(y))

if isinstance(y, set):
    song_id = ", ".join(str(item) for item in y)

# Assuming `audio` is the variable holding your audio bytes

# Ensure `x` contains 'bytes' as key
audio_bytes = x['bytes']  # Extract the audio bytes

# Create an Audio object using the byte data
audio = Audio(data=audio_bytes, rate=44100)  # Replace 44100 with the actual sample rate of your audio data

# Save the byte data to a .wav file
with open(f'/Users/Malek/Downloads/music_dataset_separe/Folk/song{song_id}.wav', 'wb') as f:
    f.write(audio_bytes)

# Display the audio
Audio(audio_bytes, rate=44100)  # Replace 44100 with the correct sampling rate


37
<class 'numpy.int64'>


In [ ]:
from IPython.display import Audio
import os

# Directory to save audio files
output_dir = '/Users/Malek/Downloads/music_dataset_separe/Folk'
os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    # Extract 'bytes' and other relevant data
    x = row[0]  # Assuming the first column contains a dictionary with 'bytes'
    y = row[1]  # Assuming the second column contains additional metadata like song_id

    print(y)
    print(type(y))

    # Handle 'y' if it is a set (e.g., song_id)
    if isinstance(y, set):
        song_id = "_".join(str(item) for item in y)  # Join the set items to create a unique identifier
    else:
        song_id = str(y)  # Use `y` directly if it's not a set

    # Ensure `x` contains 'bytes' as key
    if 'bytes' not in x:
        print(f"Row {index} does not contain 'bytes', skipping...")
        continue  # Skip this row if 'bytes' key is missing

    audio_bytes = x['bytes']  # Extract the audio bytes

    # Save the byte data to a .wav file
    output_path = os.path.join(output_dir, f'song{song_id}.wav')
    with open(output_path, 'wb') as f:
        f.write(audio_bytes)

    # Display the audio (optional in a loop; comment out if not required)
    display(Audio(audio_bytes, rate=44100))  # Replace 44100 with the correct sampling rate


Enregistrer chaque audio d'un genre dans le dossier correspondant


In [ ]:
import pandas as pd
import wave

# Assuming 'audio' column contains raw PCM binary data for .wav files
audio_column = 'audio'

# Iterate through all records in the DataFrame
for index, row in df.iterrows():
    # Extract the binary audio data
    audio_data = df.iloc[0, 0]['bytes']

    # Create a filename for the .wav file
    filename = f"folk_audio_{index + 1}.wav"

    # Write the binary data to a .wav file
    with wave.open(filename, 'wb') as wf:
        # Set parameters (adjust as needed based on your audio data properties)
        wf.setnchannels(1)       # Mono audio (adjust if stereo)
        wf.setsampwidth(2)       # 2 bytes per sample (16-bit PCM)
        wf.setframerate(44100)   # 44.1 kHz sampling rate
        wf.writeframes(audio_data)

    print(f"Saved {filename}")


Enregister les FDP d'audio en .mp3


In [ ]:
import pandas as pd
import lameenc
import io

# Assuming 'audio' column contains raw PCM binary data for .wav files
audio_column = 'audio'

# Function to encode raw PCM data to MP3
def encode_to_mp3(pcm_data, sample_rate=44100, channels=1, sample_width=2):
    encoder = lameenc.Encoder()
    encoder.set_bit_rate(128)  # Set bitrate (e.g., 128 kbps)
    encoder.set_in_sample_rate(sample_rate)
    encoder.set_channels(channels)
    encoder.set_quality(2)  # Set quality (2 = high quality)
    return encoder.encode(pcm_data) + encoder.flush()

# Iterate through all records in the DataFrame
for index, row in df.iterrows():
    # Extract the binary audio data
    audio_data = row[audio_column]['bytes']

    # Encode the raw PCM data to MP3
    mp3_data = encode_to_mp3(audio_data, sample_rate=44100, channels=1, sample_width=2)

    # Create a filename for the .mp3 file
    filename = "/Users/Malek/Downloads/music_dataset_separe/Folk f'folk_audio_{index + 1}.mp3"

    # Write the MP3 data to a file
    with open(filename, 'wb') as mp3_file:
        mp3_file.write(mp3_data)

    print(f"Saved {filename}")


ModuleNotFoundError: No module named 'lameenc'

Transformation en MEL-Spectrogramme

AVEC SCIPY

In [ ]:
!pip install scipy matplotlib numpy


In [ ]:
import numpy as np
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt

# Function to create Mel filter banks
def mel_filter_bank(num_filters, fft_size, sample_rate, fmin=0, fmax=None):
    if fmax is None:
        fmax = sample_rate / 2  # Nyquist frequency

    # Convert frequencies to Mel scale
    def hz_to_mel(hz):
        return 2595 * np.log10(1 + hz / 700.0)

    def mel_to_hz(mel):
        return 700 * (10**(mel / 2595.0) - 1)

    mel_min = hz_to_mel(fmin)
    mel_max = hz_to_mel(fmax)

    # Create linearly spaced Mel points and convert to Hz
    mel_points = np.linspace(mel_min, mel_max, num_filters + 2)
    hz_points = mel_to_hz(mel_points)

    # Map Hz points to FFT bins
    bin_points = np.floor((fft_size + 1) * hz_points / sample_rate).astype(int)

    # Create filter bank
    filter_bank = np.zeros((num_filters, int(fft_size / 2) + 1))
    for i in range(1, num_filters + 1):
        filter_bank[i - 1, bin_points[i - 1]: bin_points[i]] = np.linspace(0, 1, bin_points[i] - bin_points[i - 1])
        filter_bank[i - 1, bin_points[i]: bin_points[i + 1]] = np.linspace(1, 0, bin_points[i + 1] - bin_points[i])

    return filter_bank

#Test on the first audio

x=df.iloc[0,0]
print(x['path'])

audio_path=x['path']

# Load the audio file
#sample_rate, audio = wav.read(audio_path)  # Replace with your file
sample_rate, audio = wav.read("/Users/Malek/Downloads/téléchargement.wav")  # Replace with your file

if audio.ndim > 1:
    audio = audio.mean(axis=1)  # Convert to mono

# Parameters
fft_size = 2048
hop_size = 512
num_mel_filters = 128

# Calculate STFT
def stft(signal, fft_size, hop_size):
    window = np.hanning(fft_size)
    num_frames = 1 + (len(signal) - fft_size) // hop_size
    frames = np.stack([signal[i * hop_size:i * hop_size + fft_size] for i in range(num_frames)])
    windowed_frames = frames * window
    stft_matrix = np.fft.rfft(windowed_frames, n=fft_size)
    return np.abs(stft_matrix) ** 2

spectrogram = stft(audio, fft_size, hop_size)

# Create Mel filter bank and apply
mel_filters = mel_filter_bank(num_mel_filters, fft_size, sample_rate)
mel_spectrogram = np.dot(mel_filters, spectrogram.T)

# Convert to dB
mel_spectrogram_db = 10 * np.log10(np.maximum(mel_spectrogram, 1e-10))

# Plot the Mel spectrogram
plt.figure(figsize=(10, 4))
plt.imshow(mel_spectrogram_db, aspect='auto', origin='lower',
           extent=[0, len(audio) / sample_rate, 0, sample_rate / 2])
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.tight_layout()
plt.show()


None


ValueError: File format b'OggS' not understood. Only 'RIFF', 'RIFX', and 'RF64' supported.